# Chapter 3 发现群组
物以类聚，人以群分，这里我们想学习如何利用事物间的联系而将它们进行分类。**数据聚类（data clustering）**是一种用以寻找紧密相关的事、人或观点，并将其可视化的方法。   
在我们生物领域，经常把基因根据表达量信息来进行聚类，聚类得到的基因往往在生物学上有非常重要的意义。书中第一个例子是对博客用户所讨论的话题，以及他们使用的特殊词汇进行考查，根据文字的内容及用法分别进行分级；第二个例子是根据人们拥有和希望拥有的物品信息来对人群进行分类。
开始前，介绍了**监督学习**和**无监督学习**的概念。**监督学习法（supervised learning methods）**是利用样本输入和期望输入来学习如何预测的方法，代表的有培养出网络、决策树、向量支持机、贝叶斯过滤等。**无监督学习（unsupervised learning）**也需要样品输入来进行预测，不过这里我们没有期望输出结果是怎样的，相反我们需要在一组数据中找出某种结构，适合进行探索性发现。有无监督这里可以简单看成对结果是否有相应的期望结果。无监督学习的包括聚类，负矩阵因式分解和自组织映射。
## 3.1 博客用户进行分类
在对博客用户进行分类时，我们分析这些博客用户的标题内容，根据内容的相似性进行分析。分析前，首先需要把这些文本信息转换成对应的数值型数据，这里书里采用的是分别统计单词在各个博客内容中出现的频数。目录中已经有文件`blogdata.txt`存储了词频信息，可以直接读取。详细获取过程可以参考书上内容和`generatefeedvector.py`。  
聚类时先从单一元素开始（这里的元素可以看成初始群组），分别计算元素之间的联系，再把距离最近的两个元素当成一个群组。第二次计算前面得到的群组之间的联系（新组的值为旧组中值的平均值），再把距离相近的群组进行合并。一直循环，直到最后得到一个群组为止。这个过程可以利用**树状图**来进行展示。

In [1]:
def readfile(filename):
    with open(filename) as handle:
        lines = handle.readlines()
    # 列名
    colnames = lines[0].strip().split('\t')[1:]
    rownames = []
    data = []
    for line in lines[1:]:
        p = line.strip().split('\t')
        rownames.append(p[0])
        data.append([float(x) for x in p[1:]])
    return rownames, colnames, data

In [2]:
# 计算Pearson相关性系数
from math import sqrt
def pearson(v1, v2):
    sum1 = sum(v1)
    sum2 = sum(v2)
    
    sum1Sq = sum([pow(v,2) for v in v1])
    sum2Sq = sum([pow(v,2) for v in v2])
    
    pSum = sum([v1[i]*v2[i] for i in range(len(v1))])
    
    num = pSum - (sum1*sum2/len(v1))
    den = sqrt((sum1Sq-pow(sum1,2)/len(v1))*(sum2Sq-pow(sum2,2)/len(v1)))
    if den == 0:
        return 0
    return 1.0 - num/den


In [8]:
# 描述聚类结构的类
class bicluster:
    def __init__(self, vec, left=None, right=None, distance=0.0, id=None):
        self.left = left
        self.right = right
        self.vec = vec
        self.id = id
        self.distance = distance
        

In [14]:
def hcluster(rows, distance=pearson):
    distances = {}
    currentclustid = -1
    
    clust = [bicluster(j, id=i) for i,j in enumerate(rows)]
    
    while len(clust) > 1:
        lowestpair = (0,1)
        closest = pearson(clust[0].vec, clust[1].vec)
        
        for i in range(len(clust)):
            for j in range(i+1, len(clust)):
                # 书中这里pearson写成distance，需要注意一下
                if (clust[i].id, clust[j].id) not in distances:
                    d = pearson(clust[i].vec, clust[j].vec)
                    distances[(clust[i].id, clust[j].id)] = d
                d = distances[(clust[i].id, clust[j].id)]
                
                if d < closest:
                    closest = d
                    lowestpair = (i,j)
        
        mergevec = [
            (clust[lowestpair[0]].vec[i]+ clust[lowestpair[1]].vec[i])/2.0  
            for i in range(len(clust[0].vec))
        ]
        
        newcluster = bicluster(mergevec, left=clust[lowestpair[0]],
                               right = clust[lowestpair[1]],
                               distance=closest, 
                               id = currentclustid
                               )
        
        currentclustid -= 1
        del clust[lowestpair[1]]
        del clust[lowestpair[0]]
        clust.append(newcluster)
    
    return clust[0]

In [4]:
blognames, words, data = readfile('blogdata.txt')

In [15]:
clust = hcluster(data)

In [21]:
def printclust(clust, labels=None, n=0):
    for i in range(n):
        print " ",
    if clust.id < 0:
        print '-'
    else:
        if labels==None:
            print clust.id
        else:
            print labels[clust.id]
    
    # 在函数内部调用自身，形成递归。clust的结构也是一种递归的结果
    if clust.left !=None:
        printclust(clust.left, labels=labels, n=n+1)
    if clust.right != None:
        printclust(clust.right, labels=labels, n=n+1)


In [22]:
printclust(clust, labels=blognames)

-
  gapingvoid: "cartoons drawn on the back of business cards"
  -
    -
      Schneier on Security
      Instapundit.com
    -
      The Blotter
      -
        -
          MetaFilter
          -
            SpikedHumor
            -
              Captain's Quarters
              -
                Michelle Malkin
                -
                  -
                    NewsBusters.org - Exposing Liberal Media Bias
                    -
                      -
                        Hot Air
                        Crooks and Liars
                      -
                        Power Line
                        Think Progress
                  -
                    Andrew Sullivan | The Daily Dish
                    -
                      Little Green Footballs
                      -
                        Eschaton
                        -
                          Talking Points Memo: by Joshua Micah Marshall
                          Daily Kos
        -
          43 Folders
 

In [24]:
clust.id, clust.left, clust.right

(-98,
 <__main__.bicluster instance at 0x7f44df6b1488>,
 <__main__.bicluster instance at 0x7f44d83be8c0>)

## 3.2 对聚类结果进行可视化
上面虽然已经打印出了博客聚类的结果，不过还不够直观。书上又重新想画一个树状图来展示聚类结果。这里利用的PIL（Python Image Library）模块，可以尝试用`pip install PIL`进行安装。  
我们需要在一个将聚类结果画在一个图上，首先需要获得聚类的总体高度，如果这个聚类没有分支，则高度为1，其他为各分支高度之和。

In [25]:
def getheight(clust):
    if clust.left == None and clust.right == None:
        return 1
    return getheight(clust.left) + getheight(clust.right)

In [26]:
getheight(clust)

99

In [27]:
def getdepth(clust):
    if clust.left == None and clust.right == None:
        return 0.0
    return max(getdepth(clust.left), getdepth(clust.right)) + clust.distance

In [29]:
getdepth(clust), clust.distance

(23.188162327170954, 0.9960012161819306)

In [30]:
from PIL import Image, ImageDraw

In [31]:
def drawnode(draw, clust, x, y, scaling, labels):
    if clust.id < 0:
        h1 = getheight(clust.left) * 20
        h2 = getheight(clust.right) * 20
        top = y - (h1+h2)/2
        bottom = y+(h1+h2)/2
        # 线的长度
        ll = clust.distance*scaling
        # 聚类到其子节点的垂直线
        draw.line((x, top + h1/2, x, bottom-h2/2), fill=(255,0,0))
        # 连接左侧节点的水平线
        draw.line((x, top+h1/2, x+ll, top+h1/2), fill=(255,0,0))
        # 连接右侧节点的水平线
        draw.line((x, bottom-h2/2, x+ll, bottom-h2/2), fill=(255,0,0))
        
        # 调用函数绘制左右节点
        drawnode(draw, clust.left, x+ll, top+h1/2, scaling, labels)
        drawnode(draw, clust.right, x+ll, bottom-h2/2, scaling, labels)
    else:
        draw.text((x+5, y-7), labels[clust.id], (0,0,0))
        
        

In [32]:
def drawdendrogram(clust, labels, jpeg='clusters.jpg'):
    # 这里设置了图片的长宽，depth相当于x轴方向，表示距离的大小
    h = getheight(clust) * 20
    w = 1200
    depth = getdepth(clust)
    
    scaling = float(w-150)/depth
    
    img = Image.new('RGB', (w,h), (255,255,255))
    draw = ImageDraw.Draw(img)
    
    draw.line((0, h/2, 10, h/2), fill=(255,0,0))
    
    drawnode(draw, clust, 10, (h/2), scaling, labels)
    img.save(jpeg, 'JPEG')

In [33]:
drawdendrogram(clust, blognames, jpeg="blogclust.jpg")

## 3.3 对列聚类
上面我们对博客进行了聚类，这里我们换一个方向，置换一下数据，对出现的词汇进行聚类，看会有什么结果

In [39]:
def rotatematrix(data):
    return [[data[i][j] for i in range(len(a))] for j in range(len(data[0]))]

In [43]:
rdata = rotatematrix(data)
wordclust = hcluster(rdata)
drawdendrogram(wordclust, labels=words, jpeg='wordclust.jpg')

## 3.4 K-means均值聚类  
层次聚类的方法有两个缺点：
1. 在没有额外投入的情况下，树形图是不会真正将数据分成不同的分组  
2. 需要计算两两之间的相似性，计算量比较大    

这里K-means聚类就可以避免上面的情况了，在计算前，我们会提前告知需要分成几个小组，然后程序会随机先生成*k*个中心位置，然后将各个数据点分配到最临近的中心点，之后再重新计算中心位置，之后又重新进行新一轮的分配，直到分配结果不会发生变化为止。


In [ ]:
import random
def kcluster(rows, distance=pearson, k=4):
    ranges = [(min([row[i] for row in rows]), max([row[i] for row in rows]))  
            for i in range(len(rows[0]))]
    clusters = [[random.random()*(ranges[i][1]-ranges[i][0]) + ranges[i][0]
                for i in range(len(rows[0]))] 
               for j in range(k)]
    lastmatches = None
    for t in range(100):
        print "Iteration %d" % t
        bestmatches = [[] for i in range(k)]
    
    for j in range(len(rows)):
        row = rows[j]
        bestmatch = 0
        for i in range(k):
            d = distance(clusters[i], row)
            if d < distance(clusters[bestmatch], row):
                bestmatch = i
        bestmatches[bestmatch].append(j)
        
    if bestmatches == lastmatches:
        break
    
    lastmatches = bestmatches
    
    for i in range(k):
        avgs = [0,0] * len(rows[0])
        